In [166]:
## Libraries
import pandas as pd
import numpy as np
import sklearn as sk
import datetime as dt
import pickle
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
import pyarrow
import mlflow
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import argparse
import os
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import argparse
import mlflow
from hyperopt import hp, space_eval
from hyperopt.pyll import scope
from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error




In [307]:
## Answer 1
!mlflow --version

mlflow, version 1.26.0


In [3]:
## Import Trip Data Green Records January, February, March
db = pd.read_parquet("green01.parquet")
db2 = pd.read_parquet("green02.parquet")
db3 = pd.read_parquet("green03.parquet")

In [4]:
## January data
db['PULocationID']=db['PULocationID'].replace(np.nan, -1)
db['DOLocationID']=db['DOLocationID'].replace(np.nan, -1)
categorical = ['PULocationID', 'DOLocationID']
db[categorical] = db[categorical].astype(str)
db ["duration"] = (db.lpep_dropoff_datetime - db.lpep_pickup_datetime) / pd.Timedelta('1 minute')
db = db[db['duration'] <= 60]


In [5]:
## Clean january data and create some variables 
def preprocess(db: pd.DataFrame, dv: DictVectorizer, fit_dv: bool = False):
    db['PU_DO'] = db['PULocationID'] + '_' + db['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = db[categorical + numerical].to_dict(orient='records')
    if fit_dv:
        X = dv.fit_transform(dicts)
    else:
        X = dv.transform(dicts)
    return X, dv

In [6]:
## February data
db2['PULocationID']=db2['PULocationID'].replace(np.nan, -1)
db2['DOLocationID']=db2['DOLocationID'].replace(np.nan, -1)
categorical = ['PULocationID', 'DOLocationID']
db2[categorical] = db2[categorical].astype(str)
db2 ["duration"] = (db2.lpep_dropoff_datetime - db2.lpep_pickup_datetime) / pd.Timedelta('1 minute')
db2 = db2[db2['duration'] <= 60]

In [7]:
## Clean february
#  data and create some variables 
def preprocess(db2: pd.DataFrame, dv: DictVectorizer, fit_dv: bool = False):
    db2['PU_DO'] = db2['PULocationID'] + '_' + db2['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = db2[categorical + numerical].to_dict(orient='records')
    if fit_dv:
        X = dv.fit_transform(dicts)
    else:
        X = dv.transform(dicts)
    return X, dv

In [8]:
## March data
db3['PULocationID']=db3['PULocationID'].replace(np.nan, -1)
db3['DOLocationID']=db3['DOLocationID'].replace(np.nan, -1)
categorical = ['PULocationID', 'DOLocationID']
db3[categorical] = db3[categorical].astype(str)
db3 ["duration"] = (db3.lpep_dropoff_datetime - db3.lpep_pickup_datetime) / pd.Timedelta('1 minute')
db3 = db3[db3['duration'] <= 60]

In [9]:
## Clean march data and create some variables 
def preprocess(db3: pd.DataFrame, dv: DictVectorizer, fit_dv: bool = False):
    db3['PU_DO'] = db3['PULocationID'] + '_' + db3['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = db3[categorical + numerical].to_dict(orient='records')
    if fit_dv:
        X = dv.fit_transform(dicts)
    else:
        X = dv.transform(dicts)
    return X, dv

In [10]:
## Create dependent variable (duration)
target = 'duration'
y_train = db[target].values
y_valid = db2[target].values
y_test = db3[target].values

In [21]:
## Vectorize the data sets    
dv = DictVectorizer()
X_train, dv = preprocess(db, dv, fit_dv=True)
X_valid, _ = preprocess(db2, dv, fit_dv=False)
X_test, _ = preprocess(db3, dv, fit_dv=False)

In [23]:
## Determine the space in the computer where are gping to be the data
os.makedirs('/Users/agvg/Documents/DS/MLOPS/', exist_ok=True)

In [ ]:
## Define dump_pickle
def dump_pickle(obj, filename):
    with open(filename, "wb") as f_out:
        return pickle.dump(obj, f_out)

In [ ]:
    ## Save dictvectorizer and datasets
    # Answer 2 = 4 packages   
dump_pickle(dv, os.path.join('/Users/agvg/Documents/DS/MLOPS/', "dv.pkl"))
dump_pickle((X_train, y_train), os.path.join('/Users/agvg/Documents/DS/MLOPS/', "train.pkl"))
dump_pickle((X_valid, y_valid), os.path.join('/Users/agvg/Documents/DS/MLOPS/', "valid.pkl"))
dump_pickle((X_test, y_test), os.path.join('/Users/agvg/Documents/DS/MLOPS/', "test.pkl"))


In [40]:
## Load_Pickle 
def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)

In [129]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_valid, label=y_valid)

In [164]:
## Question 3 Train data_set
# Answer 17 parameters
mlflow.autolog()

with mlflow.start_run():
                mlflow.set_tag("model","alexander")
                rf = RandomForestRegressor(max_depth=10, random_state=0)
                rf.fit(X_train, y_train)
                y_pred = rf.predict(X_valid)
                rmse = mean_squared_error(y_valid, y_pred, squared=False)
                mlflow.log_metric("rmse",rmse)




2022/05/28 20:24:08 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/05/28 20:24:10 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


In [ ]:
## Question 5 valid test
## Answer 4 = 

mlflow.autolog()

with mlflow.start_run():
 def objective(params):

        rf = RandomForestRegressor(**params)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_valid)
        rmse = mean_squared_error(y_valid, y_pred, squared=False)

        return {'loss': rmse, 'status': STATUS_OK}

 search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
        'n_estimators': scope.int(hp.quniform('n_estimators', 10, 50, 1)),
        'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
        'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 4, 1)),
        'random_state': 42
    }

 rstate = np.random.default_rng(42)  # for reproducible results
fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=50,
        trials=Trials(),
        rstate=rstate
    )

In [205]:
## Question 6 Lauch the tracking server
# Answer = 6,55 
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi', tags={}>,
 <Experiment: artifact_location='./mlruns/4', experiment_id='4', lifecycle_stage='active', name='Random-forest-2', tags={}>,
 <Experiment: artifact_location='./mlruns/5', experiment_id='5', lifecycle_stage='active', name='random-forest-best-models', tags={}>]

In [261]:
client.create_experiment(name='Test/NYC/Taxi')

'6'

In [268]:
from mlflow.entities import ViewType

runs= client.search_runs(
experiment_ids='4',
filter_string="",
run_view_type=ViewType.ACTIVE_ONLY,
max_results=5,
order_by=["metrics.rmse ASC"]
)

In [267]:
import mlflow 
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [297]:
run_id = '23b2baeb18d24287b03440611051c6ba'
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc_taxi_regressor")

Registered model 'nyc_taxi_regressor' already exists. Creating a new version of this model...
2022/05/29 15:07:29 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc_taxi_regressor, version 2
Created version '2' of model 'nyc_taxi_regressor'.


<ModelVersion: creation_timestamp=1653858449024, current_stage='None', description=None, last_updated_timestamp=1653858449024, name='nyc_taxi_regressor', run_id='23b2baeb18d24287b03440611051c6ba', run_link=None, source='./mlruns/4/23b2baeb18d24287b03440611051c6ba/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [270]:
model_uri

'runs:/23b2baeb18d24287b03440611051c6ba/model'

In [274]:
model_name="nyc_taxi_regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.current_stage}, stage: {version.current_stage}")

version: None, stage: None


In [277]:
model_version = 1
new_stage= "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1653856864690, current_stage='Staging', description=None, last_updated_timestamp=1653857417096, name='nyc_taxi_regressor', run_id='23b2baeb18d24287b03440611051c6ba', run_link=None, source='./mlruns/4/23b2baeb18d24287b03440611051c6ba/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [278]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}")

<ModelVersion: creation_timestamp=1653856864690, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2022-05-29', last_updated_timestamp=1653857420584, name='nyc_taxi_regressor', run_id='23b2baeb18d24287b03440611051c6ba', run_link=None, source='./mlruns/4/23b2baeb18d24287b03440611051c6ba/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [309]:
data_path='/Users/agvg/Documents/DS/MLOPS'
params = '/Users/agvg/Documents/DS/MLOPS/MLOPS/artifacts'

In [290]:
df = pd.read_parquet("https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-03.parquet")

In [300]:
preprocessor= '/Users/agvg/Documents/DS/MLOPS/MLOPS/models'

In [301]:

client.download_artifacts(run_id=run_id, path=preprocessor, dst_path='.')

'/Users/agvg/Documents/DS/MLOPS/MLOPS/models'

In [304]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 486 ms, sys: 94.5 ms, total: 580 ms
Wall time: 753 ms


{'rmse': 6.672409854377293}

In [306]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1653856864690, current_stage='Production', description='The model version 1 was transitioned to Staging on 2022-05-29', last_updated_timestamp=1653858726065, name='nyc_taxi_regressor', run_id='23b2baeb18d24287b03440611051c6ba', run_link=None, source='./mlruns/4/23b2baeb18d24287b03440611051c6ba/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>